# Distance Matrix
This code takes as inputs a collection of shape files of the different districts of Greater Mexico City and creates the diatance matrix in which each row and each column correspond to a district and each entry correspond to the distance between the centroid of the two regions.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import geopy.distance

### Preprocessing

In [2]:
# Read shapefile with the different districts
gdf = gpd.GeoDataFrame.from_file('../data/shapes/DistritosEODHogaresZMVM2017.shp')

# Create new useful variables
gdf['centroids'] = gdf['geometry'].apply(lambda x: tuple(x.centroid.coords)[0])
gdf['area'] = gdf['geometry'].apply(lambda x: x.area / 1000000)
gdf['l_caract'] = np.sqrt(gdf['area'])

# Convert projected coordinates into longitude and latitude
gdf.crs = {'init' :'epsg:32614'}
gdf.to_crs(epsg=4326, inplace=True)
gdf['centroids'] = gdf['geometry'].apply(lambda x: tuple(x.centroid.coords)[0])

gdf.head()

,Distrito,Descripcio,geometry,centroids,area,l_caract
0,001,Centro HistÃ³rico,POLYGON ((-99.13615639445072 19.44077855087982...,"(-99.13739953601235, 19.433368365353914)",5.572718,2.360660
1,002,Buenavista-Reforma,"POLYGON ((-99.1428980497852 19.46364494054671,...","(-99.15831812207979, 19.44277770368488)",9.149588,3.024829
2,003,Tlatelolco,POLYGON ((-99.13141865172925 19.46378321641288...,"(-99.13524802237589, 19.452623298012657)",5.187414,2.277590
3,004,Morelos,POLYGON ((-99.10946106140032 19.45291794602026...,"(-99.11631075981163, 19.436332893340886)",6.276365,2.505267
4,005,"Moctezuma, Terminal de Autobuses de Oriente (T...",POLYGON ((-99.10055671090755 19.44982753391222...,"(-99.10059106098882, 19.437499761059755)",7.688278,2.772774


### Matrix Generation

In [4]:
n = len(gdf)
dist_cell = np.ones((n, n))

for i in range(n):
    for j in range(i,n):
        dist_cell[i,j] = geopy.distance.geodesic(tuple(reversed(gdf['centroids'][i])), tuple(reversed(gdf['centroids'][j]))).km
        dist_cell[j,i] = dist_cell[i,j]


In [5]:
np.savetxt('../data/dist_cell.txt', dist_cell)

### Characteristic Length

The characteristic length of the system is needed to calibrate the extended radiation model.

In [3]:
mean_l_caract = gdf['l_caract'].median()
mean_l_caract

3.1295916904198156